# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nyirbator,47.8333,22.1333,33.42,88,100,6.33,HU,1740542963
1,1,yanbu,24.0891,38.0637,68.04,40,40,6.91,SA,1740542965
2,2,bandon,51.7469,-8.7425,42.39,84,7,11.90,IE,1740542967
3,3,puerto natales,-51.7236,-72.4875,41.92,70,30,12.10,CL,1740542670
4,4,gulf shores,30.2460,-87.7008,60.03,85,0,5.75,US,1740542763


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
#%%capture --no-display

# Configure the map plot
humidityPlot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.7,
    tiles = "OSM",
    frame_height = 504,
    frame_width = 896,
    hover_cols = ["City", "Humidity"]
    
)

# Display the map
humidityPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [32]:
# Narrow down cities that fit criteria and drop any results with null values - 
# max temp: 27 C ~ 80.6 F
# min temp: 21 C ~ 69.8 F
# wind < 4.5
# cloudiness = 0

idealCityDF = city_data_df.loc[
    (city_data_df["Max Temp"] > 69.8) & (city_data_df["Max Temp"] < 80.6) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
idealCityDF = idealCityDF.dropna()

# Display sample data
print(len(idealCityDF))
idealCityDF.head()

5


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,san luis de la loma,17.2706,-100.8939,75.15,83,0,3.91,MX,1740542994
15,15,lazaro cardenas,17.9583,-102.2000,75.11,80,0,4.12,MX,1740542996
185,185,victorica,-36.2151,-65.4359,70.68,77,0,3.36,AR,1740543381
205,205,ixtapa,20.7000,-105.2000,74.23,83,0,2.30,MX,1740543430
545,545,san rafael,-34.6177,-68.3301,74.52,66,0,1.74,AR,1740544177


### Step 3: Create a new DataFrame called `hotel_df`.

In [34]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealCityDF[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
14,san luis de la loma,MX,17.2706,-100.8939,83,
15,lazaro cardenas,MX,17.9583,-102.2000,80,
185,victorica,AR,-36.2151,-65.4359,77,
205,ixtapa,MX,20.7000,-105.2000,83,
545,san rafael,AR,-34.6177,-68.3301,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [36]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude}, {latitude}, {radius}"
    params["bias"] = f"proximity:{longitude}, {latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
# filter out where there are no hotels
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "no hotel found"]
hotel_df

Starting hotel search
san luis de la loma - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: No hotel found
victorica - nearest hotel: No hotel found
ixtapa - nearest hotel: No hotel found
san rafael - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
14,san luis de la loma,MX,17.2706,-100.8939,83,No hotel found
15,lazaro cardenas,MX,17.9583,-102.2000,80,No hotel found
185,victorica,AR,-36.2151,-65.4359,77,No hotel found
205,ixtapa,MX,20.7000,-105.2000,83,No hotel found
545,san rafael,AR,-34.6177,-68.3301,66,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [38]:
##%%capture --no-display

# Configure the map plot
HotelCountry = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = .7,
    tiles = "OSM",
    frame_height = 500,
    frame_width = 800,
    hover_cols = ["City","Hotel Name", "Humidity", "Country"]
)  
HotelCountry

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)